<!-- TABS -->
# Retrieval augmented generation

<!-- TABS -->
## Connect to superduper

:::note
Note that this is only relevant if you are running superduper in development mode.
Otherwise refer to "Configuring your production system".
:::

In [1]:
from superduper import superduper

db = superduper('mongomock:///test_db')

2024-Oct-08 14:04:03.04| INFO     | Duncans-MacBook-Pro.local| superduper.misc.plugins:13   | Loading plugin: mongodb
2024-Oct-08 14:04:03.07| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:103  | Building Data Layer
2024-Oct-08 14:04:03.07| INFO     | Duncans-MacBook-Pro.local| superduper.base.build:173  | Configuration: 
 +---------------+----------------------+
| Configuration |        Value         |
+---------------+----------------------+
|  Data Backend | mongomock:///test_db |
+---------------+----------------------+


In [9]:
COLLECTION_NAME = 'data'

<!-- TABS -->
## Get useful sample data

In [7]:
# <tab: Text>
import json

with open('data.json', 'r') as f:
    data = json.load(f)

In [ ]:
# <tab: PDF>
!curl -O https://superduperdb-public-demo.s3.amazonaws.com/pdfs.zip && unzip -o pdfs.zip
import os

data = [f'pdfs/{x}' for x in os.listdir('./pdfs') if x.endswith('.pdf')]

In [8]:
datas = [{'x': d} for d in data]

<!-- TABS -->
## Insert simple data

After turning on auto_schema, we can directly insert data, and superduper will automatically analyze the data type, and match the construction of the table and datatype.

In [10]:
from superduper import Document

ids = db.execute(db[COLLECTION_NAME].insert([Document(data) for data in datas]))

2024-Oct-08 14:10:23.61| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:365  | Table data does not exist, auto creating...
2024-Oct-08 14:10:23.61| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:371  | Creating table data with schema {('x', 'str'), ('_fold', 'str')}
2024-Oct-08 14:10:23.61| WARNING  | Duncans-MacBook-Pro.local| superduper.base.document:452  | Leaf str already exists
2024-Oct-08 14:10:23.65| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:346  | Inserted 209 documents into data


<!-- TABS -->
## Apply a chunker for search

:::note
Note that applying a chunker is ***not*** mandatory for search.
If your data is already chunked (e.g. short text snippets or audio) or if you
are searching through something like images, which can't be chunked, then this
won't be necessary.
:::

In [11]:
# <tab: Text>
from superduper import model

CHUNK_SIZE = 200

@model(flatten=True, model_update_kwargs={})
def chunker(text):
    text = text.split()
    chunks = [' '.join(text[i:i + CHUNK_SIZE]) for i in range(0, len(text), CHUNK_SIZE)]
    return chunks

In [ ]:
# <tab: PDF>
!pip install -q "unstructured[pdf]"
from superduper import model
from unstructured.partition.pdf import partition_pdf

CHUNK_SIZE = 500

@model(flatten=True)
def chunker(pdf_file):
    elements = partition_pdf(pdf_file)
    text = '\n'.join([e.text for e in elements])
    chunks = [text[i:i + CHUNK_SIZE] for i in range(0, len(text), CHUNK_SIZE)]
    return chunks

Now we apply this chunker to the data by wrapping the chunker in `Listener`:

In [12]:
from superduper import Listener

upstream_listener = Listener(
    model=chunker,
    select=db[COLLECTION_NAME].select(),
    key='x',
    uuid="chunker",
    identifier='chunker',
)

## Select outputs of upstream listener

:::note
This is useful if you have performed a first step, such as pre-computing 
features, or chunking your data. You can use this query to 
operate on those outputs.
:::

<!-- TABS -->
## Build text embedding model

In [13]:
# <tab: OpenAI>
import os
os.environ['OPENAI_API_KEY'] = 'sk-<your-secret>'
from superduper_openai import OpenAIEmbedding

embedding_model = OpenAIEmbedding(identifier='text-embedding-ada-002')

In [ ]:
# <tab: JinaAI>
import os
from superduper_jina import JinaEmbedding

os.environ["JINA_API_KEY"] = "jina_xxxx"
 
# define the model
embedding_model = JinaEmbedding(identifier='jina-embeddings-v2-base-en')

In [ ]:
# <tab: Sentence-Transformers>
!pip install sentence-transformers
from superduper import vector
import sentence_transformers
from superduper_sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer(
    identifier="embedding",
    object=sentence_transformers.SentenceTransformer("BAAI/bge-small-en"),
    datatype=vector(shape=(1024,)),
    postprocess=lambda x: x.tolist(),
    predict_kwargs={"show_progress_bar": True},
)

## Create vector-index

In [14]:
from superduper import VectorIndex, Listener

vector_index_name = 'vector-index'

vector_index = VectorIndex(
    vector_index_name,
    indexing_listener=Listener(
        key=upstream_listener.outputs,
        select=db[upstream_listener.outputs].select(),
        model=embedding_model,
        identifier='embedding-listener',
        upstream=[upstream_listener],
    )
)

In [15]:
db.apply(vector_index)

2024-Oct-08 14:15:11.75| INFO     | Duncans-MacBook-Pro.local| superduper.components.listener:94   | Requesting listener setup on CDC service
2024-Oct-08 14:15:11.75| INFO     | Duncans-MacBook-Pro.local| superduper.components.listener:104  | Skipping listener setup on CDC service since no URI is set
2024-Oct-08 14:15:11.75| INFO     | Duncans-MacBook-Pro.local| superduper.jobs.queue:210  | Running jobs for listener::chunker
2024-Oct-08 14:15:11.75| INFO     | Duncans-MacBook-Pro.local| superduper.backends.local.compute:67   | Submitting job. function:<function method_job at 0x106c06e60>
2024-Oct-08 14:15:11.76| INFO     | Duncans-MacBook-Pro.local| superduper.components.model:732  | Requesting prediction in db - [chunker] with predict_id chunker

2024-Oct-08 14:15:11.86| WARNING  | Duncans-MacBook-Pro.local| superduper.base.document:452  | Leaf ID already exists
2024-Oct-08 14:15:11.86| WARNING  | Duncans-MacBook-Pro.local| superduper.base.document:452  | Leaf str already exists
2024-

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.64s/it]


2024-Oct-08 14:15:18.55| INFO     | Duncans-MacBook-Pro.local| superduper.components.model:862  | Adding 363 model outputs to `db`
2024-Oct-08 14:15:21.01| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:346  | Inserted 363 documents into _outputs__embedding-listener
2024-Oct-08 14:15:21.02| SUCCESS  | Duncans-MacBook-Pro.local| superduper.backends.local.compute:73   | Job submitted on <superduper.backends.local.compute.LocalComputeBackend object at 0x16062be20>.  function:<function method_job at 0x106c06e60> future:444109a7-933e-45db-8064-dd537272f0d7
2024-Oct-08 14:15:21.05| INFO     | Duncans-MacBook-Pro.local| superduper.jobs.queue:210  | Running jobs for vector_index::vector-index
2024-Oct-08 14:15:21.05| INFO     | Duncans-MacBook-Pro.local| superduper.backends.local.compute:67   | Submitting job. function:<function callable_job at 0x106c070a0>
2024-Oct-08 14:15:21.06| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:153  | Initializing vector searc

Loading vectors into vector-table...: 363it [00:00, 953.68it/s]


2024-Oct-08 14:15:21.44| INFO     | Duncans-MacBook-Pro.local| superduper.components.vector_index:97   | Loaded 363 vectors into vector index succesfully
2024-Oct-08 14:15:21.95| SUCCESS  | Duncans-MacBook-Pro.local| superduper.backends.local.compute:73   | Job submitted on <superduper.backends.local.compute.LocalComputeBackend object at 0x16062be20>.  function:<function callable_job at 0x106c070a0> future:e971fd14-68d4-4b82-a34a-1b01c444166b
2024-Oct-08 14:15:21.96| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:153  | Initializing vector searcher with type in_memory
2024-Oct-08 14:15:21.96| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:164  | Using vector searcher: <class 'superduper.vector_search.in_memory.InMemoryVectorSearcher'>
2024-Oct-08 14:15:21.96| INFO     | Duncans-MacBook-Pro.local| superduper.components.vector_index:54   | Loading vectors of vector-index: 'vector-index'
2024-Oct-08 14:15:21.96| INFO     | Duncans-MacBook-Pro.local| super

Loading vectors into vector-table...: 363it [00:00, 980.35it/s] 

2024-Oct-08 14:15:22.33| INFO     | Duncans-MacBook-Pro.local| superduper.components.vector_index:97   | Loaded 363 vectors into vector index succesfully


(['dbe9ae8eb92a492882428ddd90890a88',
  '851c4c2087ce475bb9c6ee1c27c999ba',
  'ee8eb87cd3c34e0a84a3da6ea9dea620'],
 VectorIndex(identifier='vector-index', uuid='d9d4f47a1e454bc39619eed037aeaebd', upstream=None, plugins=None, cache=False, indexing_listener=Listener(identifier='embedding-listener', uuid='befa04bbe7ef48a79f02ccb54736cdea', upstream=[Listener(identifier='chunker', uuid='chunker', upstream=None, plugins=None, cache=False, key='x', model=ObjectModel(identifier='chunker', uuid='a9e49743199c4344a7a4300485dd8f88', upstream=None, plugins=None, cache=False, signature='*args,**kwargs', datatype=None, output_schema=None, flatten=True, model_update_kwargs={}, predict_kwargs={}, compute_kwargs={}, validation=None, metric_values={}, num_workers=0, serve=False, object=<function chunker at 0x1606a2440>), select=data.select(), predict_kwargs={}, predict_id='chunker')], plugins=None, cache=False, key='_outputs__chunker', model=OpenAIEmbedding(identifier='text-embedding-ada-002', uuid='104

<!-- TABS -->
## Create Vector Search Model

In [17]:
from superduper.components.model import QueryModel

item = {'_outputs__chunker': '<var:query>'}

vector_search_model = QueryModel(
    identifier="VectorSearch",
    select=db[upstream_listener.outputs].like(item, vector_index=vector_index_name, n=5).select(),
    # The _source is the identifier of the upstream data, which can be used to locate the data from upstream sources using `_source`.
    postprocess=lambda docs: [{"text": doc[vector_index.indexing_listener.outputs], "_source": doc["_source"]} for doc in docs],
    db=db
)

In [18]:
vector_search_model.predict('Tell me about vector-search')

?-outputs-chunker-like-outputs-chunker-<var:query>-vector-index-vector-index-n-5-select
-outputs-chunker-like-outputs-chunker-<var:query>-vector-index-vector-index-n-5-select
2024-Oct-08 14:24:20.60| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:930  | {}


KeyError: '_outputs__embedding-listener'

In [19]:
debug

> /Users/dodo/superduper-io/superduper/superduper/misc/special_dicts.py(301)__getitem__()
    299             return self
    300         if '.' not in key:
--> 301             return super().__getitem__(key)
    302         else:
    303             try:



ipdb>  !self


Document({'_outputs__chunker': '--- sidebar_label: Build simple select queries filename: build_simple_select_queries.md --- import Tabs from \'@theme/Tabs\'; import TabItem from \'@theme/TabItem\'; <!-- TABS --> # Build simple select queries <Tabs> <TabItem value="MongoDB" label="MongoDB" default> ```python select = db[\'<table-name>\'].select() ``` </TabItem> <TabItem value="SQL" label="SQL" default> ```python select = db[\'<table-name>\'].select() ``` </TabItem> </Tabs>', '_source': ObjectId('6705212f04115afc747a2a32'), '_fold': 'train', '_id': ObjectId('6705224f04115afc747a2b1a'), 'score': 0.7706174003563})


ipdb>  q


<!-- TABS -->
## Build LLM

In [ ]:
# <tab: OpenAI>
from superduper_openai import OpenAIChatCompletion

llm = OpenAIChatCompletion(identifier='llm', model='gpt-3.5-turbo')

In [ ]:
# <tab: Anthropic>
from superduper_anthropic import AnthropicCompletions
import os

os.environ["ANTHROPIC_API_KEY"] = "sk-xxx"

predict_kwargs = {
    "max_tokens": 1024,
    "temperature": 0.8,
}

llm = AnthropicCompletions(identifier='llm', model='claude-2.1', predict_kwargs=predict_kwargs)

In [ ]:
# <tab: vLLM>
from superduper_vllm import VllmModel

predict_kwargs = {
    "max_tokens": 1024,
    "temperature": 0.8,
}


llm = VllmModel(
    identifier="llm",
    model_name="TheBloke/Mistral-7B-Instruct-v0.2-AWQ",
    vllm_kwargs={
        "gpu_memory_utilization": 0.7,
        "max_model_len": 1024,
        "quantization": "awq",
    },
    predict_kwargs=predict_kwargs,
)

In [ ]:
# <tab: Transformers>
from superduper_transformers import LLM

llm = LLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", load_in_8bit=True, device_map="cuda", identifier="llm", predict_kwargs=dict(max_new_tokens=128))

In [ ]:
# <tab: Llama.cpp>
!huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.2-GGUF mistral-7b-instruct-v0.2.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False

from superduper_llama_cpp.model import LlamaCpp
llm = LlamaCpp(identifier="llm", model_name_or_path="mistral-7b-instruct-v0.2.Q4_K_M.gguf")

## Answer question with LLM

In [ ]:
from superduper import model
from superduper.components.graph import Graph, input_node

prompt_template = (
    "Use the following context snippets, these snippets are not ordered!, Answer the question based on this context.\n"
    "{context}\n\n"
    "Here's the question: {query}"
)

@model
def build_prompt(query, docs):
    chunks = [doc["text"] for doc in docs]
    context = "\n\n".join(chunks)
    prompt = prompt_template.format(context=context, query=query)
    return prompt

# We build a graph to handle the entire pipeline

# create a input node, only have one input parameter `query`
in_ = input_node('query')
# pass the query to the vector search model
vector_search_results = vector_search_model(query=in_)
# pass the query and the search results to the prompt builder
prompt = build_prompt(query=in_, docs=vector_search_results)
# pass the prompt to the llm model
answer = llm(prompt)
# create a graph, and the graph output is the answer
rag = answer.to_graph("rag")

By applying the RAG model to the database, it will subsequently be accessible for use in other services.

In [ ]:
from superduper import Application

app = Application(
    'rag-app',
    components=[
        upstream_listener,
        vector_index,
        vector_search_model,
        rag,
    ]
)

db.apply(app)

You can now load the model elsewhere and make predictions using the following command.

In [ ]:
rag = db.load("model", 'rag')
print(rag.predict("Tell me about superduper")[0])

## Create template

In [ ]:
from superduper import Template

template = Template('rag-template', template=app, substitutions={'docs': 'collection'})

In [ ]:
template.export('.')